### Gini Impurity: 

$$ I_G(M) = \sum_{i=1}^{M} \hat{p}_{mk} (1-\hat{p}_{mk} ) = \sum_{i=1}^{M} (\hat{p}_{mk}  - \hat{p}_{mk} ^2) = \sum_{i=1}^{M} \hat{p}_{mk}  - \sum_{i=1}^{M} \hat{p}_{mk} ^2 = 1 - \sum_{i=1}^{M} \hat{p}_{mk}^2 = \sum_{i = 1, i \neq k}^{M} \hat{p}_{mi}  \hat{p}_{mk}  $$

$$ \hat{p}_{mk} = \dfrac{1}{N_m}\sum_{x_i \in R_m} I(y_i = k)$$

### Entropy: 
$$ I_H(E) = -\sum_{k=1}^{c}\hat{p}_{mk} \log \hat{p}_{mk}  $$

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn as sk

In [2]:
zoo = pd.read_csv('zoo.csv')

In [3]:
zoo.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [81]:
def p(node, current_instance, attribute):
    s = 0.0
    for index, example in node.iterrows(): 
        if example[attribute] == current_instance[attribute]:
            s += 1
    return s / node.shape[0]

# Gets the Gini Impurity of a single node
def split_cost_for_node(node):
    impurity = 0.0
    for index, example in node.iterrows(): 
        # using p() here in this way is slow 
        pr = p(node, example, 'class_type')
        impurity += pr * (1 - pr)
    return impurity

def test_split_for_finding_optimal(from_region, f, s):
    left = pd.DataFrame()
    right = pd .DataFrame()
    
    for index, instance in from_region.iterrows():
        if instance[f] >= s:
            right = right.append(instance)
        else:
            left = left.append(instance)
        return left, right

# Get the best splitting criterion
def get_split(from_region):
    features = list(from_region)
    print(from_region)
    for f in features:
        uniq = from_region[f].unique()
        for s in uniq:
            left, right = test_split_for_finding_optimal(from_region, f, s)
            # print('feature', f, 'split', s, '\nleft', left, '\nright', right, '\n')
            
    
# Split up region
def split(region):
    pass


df = pd.DataFrame(np.arange(12).reshape(3,4),
                      columns=['A', 'B', 'C', 'D'])
get_split(df)

   A  B   C   D
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11
feature A split 0 
left Empty DataFrame
Columns: []
Index: [] 
right      A    B    C    D
0  0.0  1.0  2.0  3.0

feature A split 4 
left      A    B    C    D
0  0.0  1.0  2.0  3.0 
right Empty DataFrame
Columns: []
Index: []

feature A split 8 
left      A    B    C    D
0  0.0  1.0  2.0  3.0 
right Empty DataFrame
Columns: []
Index: []

feature B split 1 
left Empty DataFrame
Columns: []
Index: [] 
right      A    B    C    D
0  0.0  1.0  2.0  3.0

feature B split 5 
left      A    B    C    D
0  0.0  1.0  2.0  3.0 
right Empty DataFrame
Columns: []
Index: []

feature B split 9 
left      A    B    C    D
0  0.0  1.0  2.0  3.0 
right Empty DataFrame
Columns: []
Index: []

feature C split 2 
left Empty DataFrame
Columns: []
Index: [] 
right      A    B    C    D
0  0.0  1.0  2.0  3.0

feature C split 6 
left      A    B    C    D
0  0.0  1.0  2.0  3.0 
right Empty DataFrame
Columns: []
Index: []

feature C split 10 
left